In [1]:
import pandas as pd
import numpy as np
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM
from summa.summarizer import summarize
from tqdm.autonotebook import tqdm

tqdm.pandas()

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Read & Preprocess Raw Data

In [2]:
with open("../data/writingPrompts/train.wp_source") as f:
    wp_source = f.readlines()
with open("../data/writingPrompts/train.wp_target") as f:
    wp_target = f.readlines()

In [3]:
len(wp_source)
len(wp_target)

272600

272600

In [4]:
# preprocess
df_wp = pd.DataFrame({'wp_source': wp_source, 'wp_target': wp_target})

# limit to [ WP ]
df_wp = df_wp.loc[df_wp['wp_source'].str.startswith("[ WP ]")]

# clean text
df_wp['wp_source'] = df_wp['wp_source'].str.replace("[ WP ]", "", regex=False)
df_wp['wp_source'] = df_wp['wp_source'].str.strip()
df_wp['wp_target'] = df_wp['wp_target'].str.replace("<newline>", "\n", regex=False)

# keep top n stories
df_wp = df_wp.head(1000)

df_wp.shape
df_wp.head()

(1000, 2)

,wp_source,wp_target
0,You 've finally managed to discover the secret...,"So many times have I walked on ruins , the rem..."
1,"The moon is actually a giant egg , and it has ...","-Week 18 aboard the Depth Reaver , Circa 2023-..."
2,You find a rip in time walking through the all...,"I was feckin ' sloshed , mate . First time I e..."
3,For years in your youth the same imaginary cha...,"“ No , no no no ... ” She backed up and turned..."
4,"You glance at your watch 10:34 am , roughly 10...",There 's a magical moment between wakefulness ...


In [5]:
df_wp['wp_source'][2]
df_wp['wp_target'][2]

'You find a rip in time walking through the alleys . You enter it to find yourself on a metal table with surgical instruments on a chair next to you .'

"I was feckin ' sloshed , mate . First time I ever was in the Big Lemon , and I 'd found me the best feckin ' pub I could imagine , I tell ya what . So I stumble out when it was closin ' time , musta been 'round 4 o'clock in the morning , and made my way through some alleys to find the quaint little AirBnB place I 'd rented for the week . \n \n A'course , that 's how many a horror story starts , ainnit ? But it was all fun and games at first . There was this bloke I saw comin ' towards me in the alley , dark as it was with only a few lights from some apartments overhead , where the folk were still awake . At least , I thought it was a bloke , but he looked more like there was n't something right with the whole alleyway . Like it was a painting someone had gone and ripped with a knife for some reason , fecked-up as it all looked , and the cut looked sort of weird and silvery , and wavy like a heat haze comin ' from a welder 's torch . \n \n Now this thing must 've been a good foot or fi

## Generate Outline

In [6]:
def generate_outline_from_story(story_text, outline_ratio=0.3):
    outline = summarize(story_text, ratio=outline_ratio, split=True)
    outline = "\n".join(outline)
    outline = "OUTLINE: \n" + outline
    return outline

In [7]:
def add_idea_to_outline(story_idea, outline):
    outline = "IDEA: \n" + story_idea + " \n\n" + outline
    return outline

In [8]:
df_wp['wp_outline'] = df_wp['wp_target'].progress_apply(lambda text: generate_outline_from_story(text, outline_ratio=0.3))
df_wp['wp_outline_with_idea'] = df_wp.progress_apply(lambda row: add_idea_to_outline(row['wp_source'], row['wp_outline']), axis=1)

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [9]:
df_wp.head(2)

,wp_source,wp_target,wp_outline,wp_outline_with_idea
0,You 've finally managed to discover the secret...,"So many times have I walked on ruins , the rem...","OUTLINE: \nAt first I was scared , each time I...",IDEA: \nYou 've finally managed to discover th...
1,"The moon is actually a giant egg , and it has ...","-Week 18 aboard the Depth Reaver , Circa 2023-...","OUTLINE: \nI walk about the dull gray halls , ...","IDEA: \nThe moon is actually a giant egg , and..."


## Generate Prompts

In [10]:
def generate_prompt(outline_text, idea=False):
    if idea:
        prompt_text = "Generate story using following story idea and story ouline: \n\n"
    else:
        prompt_text = "Generate story using following story ouline: \n\n"
    prompt = prompt_text + outline_text
    return prompt

In [11]:
df_wp['wp_prompt'] = df_wp['wp_outline'].progress_apply(lambda x: generate_prompt(x, idea=False))
df_wp['wp_prompt_with_idea'] = df_wp['wp_outline_with_idea'].progress_apply(lambda x: generate_prompt(x, idea=True))

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [12]:
df_wp.head(2)

,wp_source,wp_target,wp_outline,wp_outline_with_idea,wp_prompt,wp_prompt_with_idea
0,You 've finally managed to discover the secret...,"So many times have I walked on ruins , the rem...","OUTLINE: \nAt first I was scared , each time I...",IDEA: \nYou 've finally managed to discover th...,Generate story using following story ouline: \...,Generate story using following story idea and ...
1,"The moon is actually a giant egg , and it has ...","-Week 18 aboard the Depth Reaver , Circa 2023-...","OUTLINE: \nI walk about the dull gray halls , ...","IDEA: \nThe moon is actually a giant egg , and...",Generate story using following story ouline: \...,Generate story using following story idea and ...


In [13]:
print(df_wp['wp_prompt'][2])

Generate story using following story ouline: 

OUTLINE: 
First time I ever was in the Big Lemon , and I 'd found me the best feckin ' pub I could imagine , I tell ya what .
At least , I thought it was a bloke , but he looked more like there was n't something right with the whole alleyway .
Now this thing must 've been a good foot or five taller 'n me , but sloshed as I am , I just take off my hat and give it a good `` how-do-you-do '' , 'cause I 'm a nice fellow and I wan na pass , y'see ?
More in surprise 'n anythin ' , really , but I keep my composure and just sorta try and pass 'm as he walks past me with his long , spindly legs as silvery as the rest of 'm , with his feet makin ' noises like he 's wearin ' cowboy boots with those thingies on 'm .
'' And the feckin ' thing stretches out one of his spindly legs and wraps it around me like a cobra and pulls me into itself .
'Cause I know you found me in your fancy cyberwebs database thing and I ca n't be three hundred years old as my 

In [14]:
# print(df_wp['wp_prompt_with_idea'][2])
df_wp['wp_prompt_with_idea'][2]

"Generate story using following story idea and story ouline: \n\nIDEA: \nYou find a rip in time walking through the alleys . You enter it to find yourself on a metal table with surgical instruments on a chair next to you . \n\nOUTLINE: \nFirst time I ever was in the Big Lemon , and I 'd found me the best feckin ' pub I could imagine , I tell ya what .\nAt least , I thought it was a bloke , but he looked more like there was n't something right with the whole alleyway .\nNow this thing must 've been a good foot or five taller 'n me , but sloshed as I am , I just take off my hat and give it a good `` how-do-you-do '' , 'cause I 'm a nice fellow and I wan na pass , y'see ?\nMore in surprise 'n anythin ' , really , but I keep my composure and just sorta try and pass 'm as he walks past me with his long , spindly legs as silvery as the rest of 'm , with his feet makin ' noises like he 's wearin ' cowboy boots with those thingies on 'm .\n'' And the feckin ' thing stretches out one of his spi

## Generate Shifted Prompts & Targets

In [15]:
df_wp['wp_prompt_shift_1'] = np.roll(df_wp['wp_prompt'], shift=1)
df_wp['wp_prompt_shift_2'] = np.roll(df_wp['wp_prompt'], shift=2)
df_wp['wp_prompt_shift_3'] = np.roll(df_wp['wp_prompt'], shift=3)

assert df_wp['wp_prompt'][0] == df_wp['wp_prompt_shift_3'][3]
assert df_wp['wp_prompt'][0] == df_wp['wp_prompt_shift_2'][2]
assert df_wp['wp_prompt'][0] == df_wp['wp_prompt_shift_1'][1]

In [16]:
df_wp['wp_prompt_with_idea_shift_1'] = np.roll(df_wp['wp_prompt_with_idea'], shift=1)
df_wp['wp_prompt_with_idea_shift_2'] = np.roll(df_wp['wp_prompt_with_idea'], shift=2)
df_wp['wp_prompt_with_idea_shift_3'] = np.roll(df_wp['wp_prompt_with_idea'], shift=3)

assert df_wp['wp_prompt_with_idea'][0] == df_wp['wp_prompt_with_idea_shift_3'][3]
assert df_wp['wp_prompt_with_idea'][0] == df_wp['wp_prompt_with_idea_shift_2'][2]
assert df_wp['wp_prompt_with_idea'][0] == df_wp['wp_prompt_with_idea_shift_1'][1]

In [17]:
df_wp['wp_target_shift_1'] = np.roll(df_wp['wp_target'], shift=1)
df_wp['wp_target_shift_2'] = np.roll(df_wp['wp_target'], shift=2)
df_wp['wp_target_shift_3'] = np.roll(df_wp['wp_target'], shift=3)

assert df_wp['wp_target'][0] == df_wp['wp_target_shift_3'][3]
assert df_wp['wp_target'][0] == df_wp['wp_target_shift_2'][2]
assert df_wp['wp_target'][0] == df_wp['wp_target_shift_1'][1]

In [18]:
df_wp.head(2)

,wp_source,wp_target,wp_outline,wp_outline_with_idea,wp_prompt,wp_prompt_with_idea,wp_prompt_shift_1,wp_prompt_shift_2,wp_prompt_shift_3,wp_prompt_with_idea_shift_1,wp_prompt_with_idea_shift_2,wp_prompt_with_idea_shift_3,wp_target_shift_1,wp_target_shift_2,wp_target_shift_3
0,You 've finally managed to discover the secret...,"So many times have I walked on ruins , the rem...","OUTLINE: \nAt first I was scared , each time I...",IDEA: \nYou 've finally managed to discover th...,Generate story using following story ouline: \...,Generate story using following story idea and ...,Generate story using following story ouline: \...,Generate story using following story ouline: \...,Generate story using following story ouline: \...,Generate story using following story idea and ...,Generate story using following story idea and ...,Generate story using following story idea and ...,I winced as an immense crash shook the room . ...,He was standing there just like I remember him...,Humanity 's solution was space-folding . The p...
1,"The moon is actually a giant egg , and it has ...","-Week 18 aboard the Depth Reaver , Circa 2023-...","OUTLINE: \nI walk about the dull gray halls , ...","IDEA: \nThe moon is actually a giant egg , and...",Generate story using following story ouline: \...,Generate story using following story idea and ...,Generate story using following story ouline: \...,Generate story using following story ouline: \...,Generate story using following story ouline: \...,Generate story using following story idea and ...,Generate story using following story idea and ...,Generate story using following story idea and ...,"So many times have I walked on ruins , the rem...",I winced as an immense crash shook the room . ...,He was standing there just like I remember him...


## Create Datasets

In [19]:
df_zs = df_wp[['wp_prompt']].copy()
df_zs_with_idea = df_wp[['wp_prompt_with_idea']].copy()

# rename to wp_prompt
df_zs_with_idea = df_zs_with_idea.rename(columns={'wp_prompt_with_idea': 'wp_prompt'})

# add STORY: tag
df_zs['wp_prompt'] = df_zs['wp_prompt'] + '\n\n' + 'STORY: \n'
df_zs_with_idea['wp_prompt'] = df_zs_with_idea['wp_prompt'] + '\n\n' + 'STORY: \n'

df_zs.head(2)
print(df_zs['wp_prompt'][0])

df_zs_with_idea.head(2)
print(df_zs_with_idea['wp_prompt'][0])

,wp_prompt
0,Generate story using following story ouline: \...
1,Generate story using following story ouline: \...


Generate story using following story ouline: 

OUTLINE: 
At first I was scared , each time I could feel my city , my current generation collapse , break into the black hole that thrives within it , I could feel humanity , the way I 'm able to feel my body..
After a few hundred years , the pattern became obvious , no longer the war and damage that would devastate me over and over again in the far past was effecting me so dominantly .
5 world wars have passed , and now they feel like a simple sickeness that would pass by every so often , I could no longer evaluate the individual human as a being of its own , the importance of mortals is merely the same as the importance of my skin cells ; They are a part of a mechanism so much more advanced , a mechanism that is so dear to my fallen heart a mechanism that I have seen fall and rise so many times , a mechanism that when lost all of which it had , had me loosing my will to live , for the first time in all of my thousands years of existence 

,wp_prompt
0,Generate story using following story idea and ...
1,Generate story using following story idea and ...


Generate story using following story idea and story ouline: 

IDEA: 
You 've finally managed to discover the secret to immortality . Suddenly , Death appears before you , hands you a business card , and says , `` When you realize living forever sucks , call this number , I 've got a job offer for you . '' 

OUTLINE: 
At first I was scared , each time I could feel my city , my current generation collapse , break into the black hole that thrives within it , I could feel humanity , the way I 'm able to feel my body..
After a few hundred years , the pattern became obvious , no longer the war and damage that would devastate me over and over again in the far past was effecting me so dominantly .
5 world wars have passed , and now they feel like a simple sickeness that would pass by every so often , I could no longer evaluate the individual human as a being of its own , the importance of mortals is merely the same as the importance of my skin cells ; They are a part of a mechanism so much mor

In [20]:
df_os = df_wp[['wp_prompt', 'wp_prompt_shift_1', 'wp_target']].copy()
df_os_with_idea = df_wp[['wp_prompt_with_idea', 'wp_prompt_with_idea_shift_1', 'wp_target']].copy()

# cleanup instructions
df_os['wp_prompt_shift_1'] = df_os['wp_prompt_shift_1'].str.replace('Generate story using following story ouline: \n\n', '')
df_os_with_idea['wp_prompt_with_idea_shift_1'] = df_os_with_idea['wp_prompt_with_idea_shift_1'].str.replace('Generate story using following story idea and story ouline: \n\n', '')

# add shifted prompts & STORY: tag
df_os['wp_prompt'] = df_os['wp_prompt'] + '\n\n' + \
                     'STORY: \n' + \
                     df_os['wp_target'] + '\n\n' + \
                     df_os['wp_prompt_shift_1'] + '\n\n' + \
                     'STORY: \n'

df_os_with_idea['wp_prompt_with_idea'] = df_os_with_idea['wp_prompt_with_idea'] + '\n\n' + \
                     'STORY: \n' + \
                     df_os_with_idea['wp_target'] + '\n\n' + \
                     df_os_with_idea['wp_prompt_with_idea_shift_1'] + '\n\n' + \
                     'STORY: \n'

# rename to wp_prompt
df_os_with_idea = df_os_with_idea.rename(columns={'wp_prompt_with_idea': 'wp_prompt'})

df_os.head(2)
print(df_os['wp_prompt'][0])

df_os_with_idea.head(2)
print(df_os_with_idea['wp_prompt'][0])

,wp_prompt,wp_prompt_shift_1,wp_target
0,Generate story using following story ouline: \...,"OUTLINE: \nShe walked past me briskly , heels ...","So many times have I walked on ruins , the rem..."
1,Generate story using following story ouline: \...,"OUTLINE: \nAt first I was scared , each time I...","-Week 18 aboard the Depth Reaver , Circa 2023-..."


Generate story using following story ouline: 

OUTLINE: 
At first I was scared , each time I could feel my city , my current generation collapse , break into the black hole that thrives within it , I could feel humanity , the way I 'm able to feel my body..
After a few hundred years , the pattern became obvious , no longer the war and damage that would devastate me over and over again in the far past was effecting me so dominantly .
5 world wars have passed , and now they feel like a simple sickeness that would pass by every so often , I could no longer evaluate the individual human as a being of its own , the importance of mortals is merely the same as the importance of my skin cells ; They are a part of a mechanism so much more advanced , a mechanism that is so dear to my fallen heart a mechanism that I have seen fall and rise so many times , a mechanism that when lost all of which it had , had me loosing my will to live , for the first time in all of my thousands years of existence 

,wp_prompt,wp_prompt_with_idea_shift_1,wp_target
0,Generate story using following story idea and ...,IDEA: \nYou ’ ve bought/acquired a very typica...,"So many times have I walked on ruins , the rem..."
1,Generate story using following story idea and ...,IDEA: \nYou 've finally managed to discover th...,"-Week 18 aboard the Depth Reaver , Circa 2023-..."


Generate story using following story idea and story ouline: 

IDEA: 
You 've finally managed to discover the secret to immortality . Suddenly , Death appears before you , hands you a business card , and says , `` When you realize living forever sucks , call this number , I 've got a job offer for you . '' 

OUTLINE: 
At first I was scared , each time I could feel my city , my current generation collapse , break into the black hole that thrives within it , I could feel humanity , the way I 'm able to feel my body..
After a few hundred years , the pattern became obvious , no longer the war and damage that would devastate me over and over again in the far past was effecting me so dominantly .
5 world wars have passed , and now they feel like a simple sickeness that would pass by every so often , I could no longer evaluate the individual human as a being of its own , the importance of mortals is merely the same as the importance of my skin cells ; They are a part of a mechanism so much mor

In [21]:
df_fs = df_wp[['wp_prompt', 'wp_prompt_shift_1', 'wp_prompt_shift_2', 'wp_prompt_shift_3', 
               'wp_target', 'wp_target_shift_1', 'wp_target_shift_2', 'wp_target_shift_3']].copy()
df_fs_with_idea = df_wp[['wp_prompt_with_idea', 'wp_prompt_with_idea_shift_1', 'wp_prompt_with_idea_shift_2', 'wp_prompt_with_idea_shift_3', 
                         'wp_target', 'wp_target_shift_1', 'wp_target_shift_2', 'wp_target_shift_3']].copy()

# cleanup instructions
df_fs['wp_prompt_shift_1'] = df_fs['wp_prompt_shift_1'].str.replace('Generate story using following story ouline: \n\n', '')
df_fs['wp_prompt_shift_2'] = df_fs['wp_prompt_shift_2'].str.replace('Generate story using following story ouline: \n\n', '')
df_fs['wp_prompt_shift_3'] = df_fs['wp_prompt_shift_3'].str.replace('Generate story using following story ouline: \n\n', '')
df_fs_with_idea['wp_prompt_with_idea_shift_1'] = df_fs_with_idea['wp_prompt_with_idea_shift_1'].str.replace('Generate story using following story idea and story ouline: \n\n', '')
df_fs_with_idea['wp_prompt_with_idea_shift_2'] = df_fs_with_idea['wp_prompt_with_idea_shift_2'].str.replace('Generate story using following story idea and story ouline: \n\n', '')
df_fs_with_idea['wp_prompt_with_idea_shift_3'] = df_fs_with_idea['wp_prompt_with_idea_shift_3'].str.replace('Generate story using following story idea and story ouline: \n\n', '')

# add shifted prompts & STORY: tag
df_fs['wp_prompt'] = df_fs['wp_prompt'] + '\n\n' + \
                     'STORY: \n' + \
                     df_fs['wp_target'] + '\n\n' + \
                     df_fs['wp_prompt_shift_1'] + '\n\n' + \
                     'STORY: \n' + \
                     df_fs['wp_target_shift_1'] + '\n\n' + \
                     df_fs['wp_prompt_shift_2'] + '\n\n' + \
                     'STORY: \n' + \
                     df_fs['wp_target_shift_2'] + '\n\n' + \
                     df_fs['wp_prompt_shift_3'] + '\n\n' + \
                     'STORY: \n'

df_fs_with_idea['wp_prompt_with_idea'] = df_fs_with_idea['wp_prompt_with_idea'] + '\n\n' + \
                     'STORY: \n' + \
                     df_fs_with_idea['wp_target'] + '\n\n' + \
                     df_fs_with_idea['wp_prompt_with_idea_shift_1'] + '\n\n' + \
                     'STORY: \n' + \
                     df_fs_with_idea['wp_target_shift_1'] + '\n\n' + \
                     df_fs_with_idea['wp_prompt_with_idea_shift_2'] + '\n\n' + \
                     'STORY: \n' + \
                     df_fs_with_idea['wp_target_shift_2'] + '\n\n' + \
                     df_fs_with_idea['wp_prompt_with_idea_shift_3'] + '\n\n' + \
                     'STORY: \n'

# rename to wp_prompt
df_fs_with_idea = df_fs_with_idea.rename(columns={'wp_prompt_with_idea': 'wp_prompt'})

df_fs.head(2)
print(df_fs['wp_prompt'][0])

df_fs_with_idea.head(2)
print(df_fs_with_idea['wp_prompt'][0])

,wp_prompt,wp_prompt_shift_1,wp_prompt_shift_2,wp_prompt_shift_3,wp_target,wp_target_shift_1,wp_target_shift_2,wp_target_shift_3
0,Generate story using following story ouline: \...,"OUTLINE: \nShe walked past me briskly , heels ...",OUTLINE: \nHe was standing there just like I r...,OUTLINE: \nThe problem is that in the folding ...,"So many times have I walked on ruins , the rem...",I winced as an immense crash shook the room . ...,He was standing there just like I remember him...,Humanity 's solution was space-folding . The p...
1,Generate story using following story ouline: \...,"OUTLINE: \nAt first I was scared , each time I...","OUTLINE: \nShe walked past me briskly , heels ...",OUTLINE: \nHe was standing there just like I r...,"-Week 18 aboard the Depth Reaver , Circa 2023-...","So many times have I walked on ruins , the rem...",I winced as an immense crash shook the room . ...,He was standing there just like I remember him...


Generate story using following story ouline: 

OUTLINE: 
At first I was scared , each time I could feel my city , my current generation collapse , break into the black hole that thrives within it , I could feel humanity , the way I 'm able to feel my body..
After a few hundred years , the pattern became obvious , no longer the war and damage that would devastate me over and over again in the far past was effecting me so dominantly .
5 world wars have passed , and now they feel like a simple sickeness that would pass by every so often , I could no longer evaluate the individual human as a being of its own , the importance of mortals is merely the same as the importance of my skin cells ; They are a part of a mechanism so much more advanced , a mechanism that is so dear to my fallen heart a mechanism that I have seen fall and rise so many times , a mechanism that when lost all of which it had , had me loosing my will to live , for the first time in all of my thousands years of existence 

,wp_prompt,wp_prompt_with_idea_shift_1,wp_prompt_with_idea_shift_2,wp_prompt_with_idea_shift_3,wp_target,wp_target_shift_1,wp_target_shift_2,wp_target_shift_3
0,Generate story using following story idea and ...,IDEA: \nYou ’ ve bought/acquired a very typica...,"IDEA: \nIn the near future , all weapons come ...",IDEA: \nEvery starfaring species has discovere...,"So many times have I walked on ruins , the rem...",I winced as an immense crash shook the room . ...,He was standing there just like I remember him...,Humanity 's solution was space-folding . The p...
1,Generate story using following story idea and ...,IDEA: \nYou 've finally managed to discover th...,IDEA: \nYou ’ ve bought/acquired a very typica...,"IDEA: \nIn the near future , all weapons come ...","-Week 18 aboard the Depth Reaver , Circa 2023-...","So many times have I walked on ruins , the rem...",I winced as an immense crash shook the room . ...,He was standing there just like I remember him...


Generate story using following story idea and story ouline: 

IDEA: 
You 've finally managed to discover the secret to immortality . Suddenly , Death appears before you , hands you a business card , and says , `` When you realize living forever sucks , call this number , I 've got a job offer for you . '' 

OUTLINE: 
At first I was scared , each time I could feel my city , my current generation collapse , break into the black hole that thrives within it , I could feel humanity , the way I 'm able to feel my body..
After a few hundred years , the pattern became obvious , no longer the war and damage that would devastate me over and over again in the far past was effecting me so dominantly .
5 world wars have passed , and now they feel like a simple sickeness that would pass by every so often , I could no longer evaluate the individual human as a being of its own , the importance of mortals is merely the same as the importance of my skin cells ; They are a part of a mechanism so much mor

In [22]:
# write to file
df_zs[['wp_prompt']].to_parquet('../data/generated_prompts/zeroshot.parquet')
df_zs_with_idea[['wp_prompt']].to_parquet('../data/generated_prompts/zeroshot_with_idea.parquet')

df_os[['wp_prompt']].to_parquet('../data/generated_prompts/oneshot.parquet')
df_os_with_idea[['wp_prompt']].to_parquet('../data/generated_prompts/oneshot_with_idea.parquet')

df_fs[['wp_prompt']].to_parquet('../data/generated_prompts/fewshot.parquet')
df_fs_with_idea[['wp_prompt']].to_parquet('../data/generated_prompts/fewshot_with_idea.parquet')